Nama : Liany Rahmanita Dewi
Grup : M-02

Import Library

In [1]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.3 MB/s eta 0:00:00


In [2]:
# Import libraries
import re
import string
from collections import Counter

import numpy as np
import pandas as pd

from google_play_scraper import app, reviews
from google_play_scraper import Sort, reviews

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
pip install imbalanced-learn

Dataset

In [4]:
# Memuat Dataset
df = pd.read_csv('dataset_hasil_scriping.csv')


In [5]:
# Memuat Dataset dan Melakukan Analisis Data Eksplorasi (EDA)
df = pd.read_csv('dataset_hasil_scriping.csv')
print("Dataset:")
print(df.head())
print("Informasi Dataset:")
print(df.info())
print("Statistik Deskriptif:")
print(df.describe())

Dataset:
                               reviewId         userName  \
0  e58bfc99-2e53-4484-a722-7bd02f3c635a  Pengguna Google   
1  de49f6ff-69dc-4c7a-92b2-c30fabecb05a  Pengguna Google   
2  51cb3416-7da7-4554-81dc-4578aec2541e  Pengguna Google   
3  b523a0ba-e07f-4038-9b14-b6d6a9e37df6  Pengguna Google   
4  bd71cae9-39eb-406e-ba4b-7ef09a24d0f5  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0                                                 ok      5              0   
1                        pengalaman di wa ga dibalas      3              0   
2                                 supaya lebih mudah      5

Data Preprocessing

In [6]:
# Menghapus baris yang tidak memiliki nilai pada kolom 'username'
df_cleaned = df[df['userName'].notna()]

# Menampilkan jumlah nilai kosong setelah pembersihan
print("Jumlah data kosong setelah membersihkan kolom 'username':")
print(df_cleaned.isna().sum())

Jumlah data kosong setelah membersihkan kolom 'username':
reviewId                    0
userName                    0
userImage                   0
content                     0
score                       0
thumbsUpCount               0
reviewCreatedVersion     2223
at                          0
replyContent            10000
repliedAt               10000
appVersion               2223
dtype: int64


In [7]:
# Definisi fungsi untuk preprocessing teks komentar
def bersihkan_teks(teks):
    teks = teks.lower()  # ubah ke huruf kecil
    teks = re.sub(r'\d+', '', teks)  # hilangkan angka
    teks = teks.translate(str.maketrans('', '', string.punctuation))  # hapus tanda baca
    return teks

# Terapkan fungsi pembersih ke kolom 'comment' dan simpan hasilnya ke kolom baru
df['komentar_bersih'] = df['content'].apply(bersihkan_teks)


Sentimen Labeling

In [8]:
# Fungsi untuk mengklasifikasikan sentimen berdasarkan polaritas
def klasifikasi_sentimen(teks):
    from textblob import TextBlob
    skor = TextBlob(teks).sentiment.polarity
    if skor > 0:
        return 'positif'
    elif skor < 0:
        return 'negatif'
    else:
        return 'netral'

# Terapkan fungsi ke kolom teks bersih untuk menghasilkan label sentimen
df['sentimen'] = df['komentar_bersih'].apply(klasifikasi_sentimen)

Preparasi data

In [9]:
# Mengubah label sentimen menjadi bentuk numerik
encoder = LabelEncoder()
y = encoder.fit_transform(df['sentimen'])
X = df['komentar_bersih']

# Membagi data menjadi data latih dan data uji dengan proporsi 80:20
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

3 percobaan skema pelatihan

Percobaan 1: SVM dengan TF-IDF(80/20 split)

In [10]:
# Transformasi teks menggunakan TF-IDF
tfidf = TfidfVectorizer()
X_train_vec = tfidf.fit_transform(X_train_raw)
X_test_vec = tfidf.transform(X_test_raw)

# Menyeimbangkan data latih dengan SMOTE
sm = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = sm.fit_resample(X_train_vec, y_train)

# Menampilkan distribusi label setelah penyeimbangan
print("Sebaran kelas setelah SMOTE:", Counter(y_train_balanced))

Sebaran kelas setelah SMOTE: Counter({np.int64(1): 7279, np.int64(2): 7279, np.int64(0): 7279})


In [11]:
# Pembagian data 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Model 1: Support Vector Machine (SVM) ===
model_svm = SVC(kernel='linear', random_state=42)
model_svm.fit(X_train_balanced, y_train_balanced)

# Prediksi menggunakan data uji
prediksi_svm = model_svm.predict(X_test_vec)

# Evaluasi performa model
akurasi_svm = accuracy_score(y_test, prediksi_svm)
print("\nHasil Evaluasi untuk SVM:")
print(f"Akurasi: {akurasi_svm:.4f}")
print(classification_report(y_test, prediksi_svm, target_names=['negatif', 'netral', 'positif']))



Hasil Evaluasi untuk SVM:
Akurasi: 0.8340
              precision    recall  f1-score   support

     negatif       0.00      0.00      0.00        20
      netral       0.90      0.92      0.91      1808
     positif       0.05      0.04      0.04       172

    accuracy                           0.83      2000
   macro avg       0.32      0.32      0.32      2000
weighted avg       0.82      0.83      0.83      2000



Percobaan 2: Random Forest dengan Word2Vec (80/20 split)

In [12]:
from sklearn.model_selection import train_test_split

# Pembagian data 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Model 2: Random Forest Classifier ===
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train_balanced, y_train_balanced)

# Melakukan prediksi pada data pengujian
prediksi_rf = model_rf.predict(X_test_vec)

# Menampilkan hasil evaluasi model
akurasi_rf = accuracy_score(y_test, prediksi_rf)
print("\nEvaluasi Kinerja Random Forest:")
print(f"Akurasi: {akurasi_rf:.4f}")
print(classification_report(y_test, prediksi_rf, target_names=['negatif', 'netral', 'positif']))



Evaluasi Kinerja Random Forest:
Akurasi: 0.8445
              precision    recall  f1-score   support

     negatif       0.00      0.00      0.00        20
      netral       0.90      0.93      0.92      1808
     positif       0.05      0.04      0.05       172

    accuracy                           0.84      2000
   macro avg       0.32      0.32      0.32      2000
weighted avg       0.82      0.84      0.83      2000



Percobaan 3: Random Forest dengan TF-IDF (70/30 split)

In [13]:
from sklearn.model_selection import train_test_split

# Pembagian data 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Ekstraksi fitur TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Pelatihan model RF
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train_vec, y_train)

# Prediksi pada data pengujian
prediksi_rf = model_rf.predict(X_test_vec)

# Evaluasi model
akurasi_rf = accuracy_score(y_test, prediksi_rf)
print("\nEvaluasi Kinerja Random Forest:")
print(f"Akurasi: {akurasi_rf:.4f}")
print(classification_report(y_test, prediksi_rf, target_names=['negatif', 'netral', 'positif']))


Evaluasi Kinerja Random Forest:
Akurasi: 0.9757
              precision    recall  f1-score   support

     negatif       1.00      0.04      0.07        26
      netral       0.98      1.00      0.99      2719
     positif       0.97      0.82      0.89       255

    accuracy                           0.98      3000
   macro avg       0.98      0.62      0.65      3000
weighted avg       0.98      0.98      0.97      3000



Inference atau testing

In [14]:
# Membandingkan performa ketiga model berdasarkan akurasi
skor_akurasi = {
    'SVM': akurasi_svm,  # Get accuracy directly using .score()
    'Random Forest': akurasi_rf,  # Changed accuracy_rf to akurasi_rf

}

# Menentukan model dengan akurasi tertinggi
model_terbaik = max(skor_akurasi, key=skor_akurasi.get)
print(f"\nModel dengan performa terbaik (akurasi tertinggi): {model_terbaik}")


Model dengan performa terbaik (akurasi tertinggi): Random Forest


In [15]:
# Membandingkan performa kedua model berdasarkan akurasi
skor_akurasi = {
    'SVM': akurasi_svm,  # Get accuracy directly using .score()
    'Random Forest': akurasi_rf,  # Use 'akurasi_rf' instead of 'accuracy_rf'

}

# Menentukan model dengan akurasi tertinggi
model_terbaik = max(skor_akurasi, key=skor_akurasi.get)
print(f"\nModel dengan performa terbaik (akurasi tertinggi): {model_terbaik}")

# Assign the name of the best model to best_model_name
best_model_name = model_terbaik

# Latih LabelEncoder pada label asli
label_encoder = LabelEncoder()
label_encoder.fit(df['sentimen'])

# Tentukan model terbaik berdasarkan nama yang telah dipilih
if best_model_name == 'SVM':
    model_terpilih = model_svm # Use the correct model variable
    prediksi_terbaik = model_svm.predict(X_test_vec) # Use predict on correct data
elif best_model_name == 'Random Forest':
    model_terpilih = model_rf # Use the correct model variable
    prediksi_terbaik = model_rf.predict(X_test_vec) # Use predict on correct data

# Buat DataFrame untuk membandingkan label asli dan hasil prediksi
hasil_klasifikasi = pd.DataFrame({
    'Label Asli': label_encoder.inverse_transform(y_test),
    'Label Prediksi': label_encoder.inverse_transform(prediksi_terbaik)
})

print("\nHasil Perbandingan Label Asli vs Prediksi:")
print(hasil_klasifikasi.head())


Model dengan performa terbaik (akurasi tertinggi): Random Forest

Hasil Perbandingan Label Asli vs Prediksi:
  Label Asli Label Prediksi
0     netral         netral
1     netral         netral
2     netral         netral
3     netral         netral
4     netral         netral


In [16]:
!pip freeze > requirements.txt